In [3]:
# install required dependencies
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip in

In [4]:
# import all the required dependencies
import kagglehub
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/divyendu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/divyendu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/divyendu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Download dataset
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

Path to dataset files: /Users/divyendu/.cache/kagglehub/datasets/tboyle10/medicaltranscriptions/versions/1
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

   

## Data processing

- We drop every other column except transcription and medical_specialty.
- We also drop any rows with empty or null transcription or medical_specialty.

- Then we drop all the classes in the excluded specialties list below. We do this as these are general terms and don't specifically map to any specialty.
- We then merge the classes with large overlaps - e.g. Neurosurgery and neurology, Neurosurgery is a subset of neurology.

In [6]:
# Drop rows with missing values in specified columns
dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

# Keep only relevant columns
dataset = dataset[['transcription', 'medical_specialty']]

# Filter medical specialties with at least 50 occurrences
specialty_counts = dataset['medical_specialty'].value_counts()
valid_specialties = specialty_counts[specialty_counts >= 50].index
dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

# Strip spaces in 'medical_specialty' column
dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

# Remove specific categories
excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

# Define category mapping to merge similar categories
category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

# Apply category mapping
dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

# Display counts for each category
for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")


Category 0: Cardiovascular / Pulmonary: 371
Category 1: ENT - Otolaryngology: 96
Category 2: Gastroenterology: 224
Category 3: Hematology - Oncology: 90
Category 4: Neurology: 317
Category 5: Obstetrics / Gynecology: 155
Category 6: Ophthalmology: 83
Category 7: Orthopedic: 355
Category 8: Pediatrics - Neonatal: 70
Category 9: Psychiatry / Psychology: 53
Category 10: Radiology: 273
Category 11: Urology: 237


## Data processing

- We clean the text and then tokenize it and then lemmatize all the words in it.

In [ ]:
from sklearn.model_selection import train_test_split

def clean_text(text):
    lemmatizer = WordNetLemmatizer() 
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text) if word not in stopwords.words('english')])
    return text

dataset['processed_transcription'] = dataset['transcription'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

## Naive Bayes model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

pipeline = Pipeline([
    ('tfidataset', TfidfVectorizer(analyzer='word', stop_words='english',ngram_range=(1,4), max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', MultinomialNB())  # Train a Naive Bayes classifier
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.6774
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.65      0.85      0.74        74
      ENT - Otolaryngology       1.00      0.68      0.81        19
          Gastroenterology       0.81      0.76      0.78        45
     Hematology - Oncology       0.57      0.22      0.32        18
                 Neurology       0.56      0.54      0.55        63
   Obstetrics / Gynecology       0.80      0.77      0.79        31
             Ophthalmology       0.92      0.71      0.80        17
                Orthopedic       0.64      0.80      0.71        71
     Pediatrics - Neonatal       0.56      0.36      0.43        14
   Psychiatry / Psychology       1.00      0.45      0.62        11
                 Radiology       0.46      0.42      0.44        55
                   Urology       0.84      0.87      0.85        47

                  accuracy                           0.68       465
                 macro avg   

## Logistic Regression Model

In [ ]:
# Implement Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Create a pipeline with TF-IDF and Logistic Regression
lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,4), 
                             max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', LogisticRegression(max_iter=1000, C=1.0, solver='liblinear', multi_class='ovr'))
])

# Train the model
lr_pipeline.fit(X_train, y_train)

# Make predictions
lr_y_pred = lr_pipeline.predict(X_test)

# Evaluate the model
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(classification_report(y_test, lr_y_pred))

/Users/divyendu/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 0.6796
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.66      0.84      0.74        74
      ENT - Otolaryngology       0.93      0.74      0.82        19
          Gastroenterology       0.73      0.82      0.77        45
     Hematology - Oncology       0.50      0.22      0.31        18
                 Neurology       0.63      0.54      0.58        63
   Obstetrics / Gynecology       0.73      0.77      0.75        31
             Ophthalmology       0.88      0.88      0.88        17
                Orthopedic       0.71      0.87      0.78        71
     Pediatrics - Neonatal       0.75      0.43      0.55        14
   Psychiatry / Psychology       1.00      0.45      0.62        11
                 Radiology       0.24      0.20      0.22        55
                   Urology       0.88      0.89      0.88        47

                  accuracy                           0.68       465
         

## CNN model

In [80]:
# %%
# Implement CNN model for text classification
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Parameters
max_features = 50000  # Max vocabulary size
maxlen = 1000  # Max sequence length
embedding_dims = 500  # Embedding dimension
batch_size = 64
epochs = 20

# Convert text to sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train_encoded)
y_test_cat = tf.keras.utils.to_categorical(y_test_encoded)

# Build the CNN model
model = tf.keras.Sequential([
    Embedding(max_features, embedding_dims, input_length=maxlen),
    Dropout(0.2),
    Conv1D(64, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    X_train_pad, 
    y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, cnn_accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f'CNN Model Accuracy: {cnn_accuracy:.4f}')

# Get predictions
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print(classification_report(y_test, y_pred_labels))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')

/Users/divyendu/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_12         │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 340ms/step - accuracy: 0.1130 - loss: 2.4454 - val_accuracy: 0.1532 - val_loss: 2.3157
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 355ms/step - accuracy: 0.2302 - loss: 2.2272 - val_accuracy: 0.2527 - val_loss: 2.1370
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 427ms/step - accuracy: 0.3589 - loss: 1.9552 - val_accuracy: 0.3737 - val_loss: 1.9131
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 404ms/step - accuracy: 0.5139 - loss: 1.6118 - val_accuracy: 0.4462 - val_loss: 1.6817
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 421ms/step - accuracy: 0.6370 - loss: 1.2626 - val_accuracy: 0.5188 - val_loss: 1.5000
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 11s 461ms/step - accuracy: 0.6931 - loss: 0.9493 - val_accuracy: 0.5161 - val_loss: 1.4352
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 489ms/step - accuracy: 0.7982 - loss: 0.7032 - val_accuracy: 0.5242 - val_loss: 1.4253
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 485ms/step - accuracy: 0.8264 - loss: 0.5571 - val_accura

## Transformers model

In [10]:
from sklearn.preprocessing import LabelEncoder
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tf_keras.optimizers.legacy import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report
import numpy as np

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Assuming y_train and y_test are the labels for the training and testing data
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Parameters
max_len = 512  # Max sequence length
batch_size = 64
epochs = 5  # Typically, you might want fewer epochs for transformer models

# Load a pre-trained BERT model and tokenizer from Hugging Face
model_name = "bert-base-uncased"  # You can change this to other models like 'bert-large-uncased' or 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenize the input texts
def encode_texts(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="tf")

X_train_encoded = encode_texts(X_train)
X_test_encoded = encode_texts(X_test)

# Fine-tune the model
transformer_model.compile(optimizer=Adam(learning_rate=2e-5), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history_transformer = transformer_model.fit(
    X_train_encoded['input_ids'], y_train_encoded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, transformer_accuracy = transformer_model.evaluate(X_test_encoded['input_ids'], y_test_encoded, verbose=0)
print(f'Transformer Model Accuracy: {transformer_accuracy:.4f}')

# Get predictions
y_pred_probs_transformer = transformer_model.predict(X_test_encoded['input_ids'])
y_pred_classes_transformer = np.argmax(y_pred_probs_transformer, axis=1)
y_pred_labels_transformer = label_encoder.inverse_transform(y_pred_classes_transformer)

# Print classification report
print(classification_report(y_test, y_pred_labels_transformer))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')
print(f'Transformer Accuracy: {transformer_accuracy:.4f}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5


: 